# 1. Setup

## 1.1. Installing and importing libraries to the collab new session (therefore new environment)

In [1]:
!python -m nltk.downloader 'punkt'
#restart the runtime once installed

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install sumy

     |████████████████████████████████| 83 kB 1.7 MB/s 
     |████████████████████████████████| 10.1 MB 7.6 MB/s 
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21711 sha256=aaecd190309d77c7da70e312558c2e4d821a4be991b19590a83968e75c3c0606
  Stored in directory: /root/.cache/pip/wheels/d4/bf/51/81d27ad638e1a6dca4f362ecc33d1e2c764b8ea7ec751b8fc1
  Created wheel for pycountry: filename=pycountry-20.7.3-py2.py3-none-any.whl size=10746883 sha256=8e7b2ba237d25a5a9778d4cf6feb923b059fc827a05ae277e45871aa44658268
  Stored in directory: /root/.cache/pip/wheels/57/e8/3f/120ccc1ff7541c108bc5d656e2a14c39da0d824653b62284c6
Successfully built breadability pycountry


In [3]:
!pip install datasets

     |████████████████████████████████| 542 kB 4.8 MB/s 
     |████████████████████████████████| 243 kB 9.8 MB/s 
     |████████████████████████████████| 76 kB 3.3 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 118 kB 15.8 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [4]:
!pip install rouge_score

In [5]:
# Importing necessary librairies
import datasets
from datasets import load_metric
from google.colab import drive
import nltk
import numpy as np
import pandas as pd
import pickle 
from sklearn.model_selection import train_test_split
import spacy
from sumy.nlp.tokenizers import Tokenizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.text_rank import TextRankSummarizer


## 1.2. Importing personal drive to file in order to load the data


In [6]:
# Mount Google Drive to this Notebook instance.
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Loading the testing dataset

In [8]:
test_dataset = pd.read_csv('/content/drive/MyDrive/Data/test_dataset.csv') #this will be used to generate predictions 
df = pd.read_pickle('/content/drive/MyDrive/Data/SummaDevDocs_preprocessed.pickle') #this will be used to compute the ROUGE scores of the generated summaries 

In [9]:
test_dataset = test_dataset.groupby('document_label').agg(list)
test_dataset = test_dataset.reset_index()

In [10]:
#creating an empty list 
documents = []

#looping though the document labels in the df_summaries 
for label in test_dataset['document_label']:
  #looping through both the indexes and the text of the original summaries in the initial dataframe
  for index, docs in zip(df.index, df['document_text']):
        #setting the condition that if the index of the actual summary in the original document equals 
        #to the label of the document in the results (i.e. summaries) dataframe 
        #append the summary to the empty list
        if label == index:
            documents.append(docs)

test_dataset['document_text'] = documents

#creataing an empty list 
reference_summaries = []

#looping though the document labels in the df_summaries 
for label in test_dataset['document_label']:
  #looping through both the indexes and the text of the original summaries in the initial dataframe
  for index, summ in zip(df.index, df['summary_text']):
        #setting the condition that if the index of the actual summary in the original document equals 
        #to the label of the document in the results (i.e. summaries) dataframe 
        #append the summary to the empty list
        if label == index:
            reference_summaries.append(summ)

test_dataset['reference_summary'] = reference_summaries

test_dataset

,document_label,sentence,label,document_text,reference_summary
0,17,[The Africa Clean Energy (ACE) Technical Assis...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",The Africa Clean Energy (ACE) Technical Assist...,The Africa Clean Energy (ACE) Technical Assist...
1,18,[DFID’s Africa Regional Department (ARD) is th...,"[0, 0, 0, 0, 0, 0, 1, 0, 0]",DFID’s Africa Regional Department (ARD) is the...,DFID has engaged Coffey to provide third party...
2,19,[The project is working with the Federal Gover...,"[0, 1, 0, 0, 0, 0, 0, 0]",The project is working with the Federal Govern...,The project is working with the Federal Govern...
3,20,[This five-year project (September 2016 – Augu...,"[0, 0, 0, 1, 0, 0, 0, 0]",This five-year project (September 2016 – Augus...,Four major inter-related causes of conflict an...
4,36,"[PROJECT OBJECTIVES:, nnThe objective of the p...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",PROJECT OBJECTIVES:\n\nThe objective of the pr...,The objective of the project is to increase ba...
...,...,...,...,...,...
569,2967,[Money Trail is a joint project initiated by F...,"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",Money Trail is a joint project initiated by Fr...,Money Trail is a joint project initiated by Fr...
570,2973,"[Gisha, Legal Center for Freedom of Movement i...","[0, 0, 0, 1, 0, 0, 0, 0]","Gisha\, Legal Center for Freedom of Movement i...",Gisha#s direct intervention with Palestinians ...
571,2976,[Legal Aid Service Providers Network Uganda (L...,"[0, 1, 1, 0, 0, 0, 0]",Legal Aid Service Providers Network Uganda (LA...,"this Voice linking and learning project, LASP..."
572,2977,[The latest findings from a countrywide season...,"[0, 0, 0, 0, 1, 0, 0, 0]",The latest findings from a countrywide seasona...,nOxfam provides integrated EFSVL (Emergency Fo...


# 3. Loading the summariser 

In [11]:
# Defining a function to parse, tokenize and summarise the text 
def summariser(doc):
  parser = PlaintextParser.from_string(doc, Tokenizer('english'))
  textrank = TextRankSummarizer()
  summary = textrank(parser.document, 2)
  return summary

# Applying the function to the dataset
#creating an empty list
text_rank_summaries = []

#looping in the texts of teh datasets
for doc in test_dataset['document_text']:
  #applying the function to each text 
  #appending the result to the list 
  text_rank_summaries.append(summariser(doc))

# Creating a column filled with the generated summaries 
df_summaries = test_dataset.copy()
df_summaries['predicted_summary'] = text_rank_summaries
df_summaries

,document_label,sentence,label,document_text,reference_summary,predicted_summary
0,17,[The Africa Clean Energy (ACE) Technical Assis...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",The Africa Clean Energy (ACE) Technical Assist...,The Africa Clean Energy (ACE) Technical Assist...,(The ACE TAF is part of the broader ACE Busine...
1,18,[DFID’s Africa Regional Department (ARD) is th...,"[0, 0, 0, 0, 0, 0, 1, 0, 0]",DFID’s Africa Regional Department (ARD) is the...,DFID has engaged Coffey to provide third party...,(This makes it difficult for ARD and JSD to as...
2,19,[The project is working with the Federal Gover...,"[0, 1, 0, 0, 0, 0, 0, 0]",The project is working with the Federal Govern...,The project is working with the Federal Govern...,(Focusing on strengthening customs operations ...
3,20,[This five-year project (September 2016 – Augu...,"[0, 0, 0, 1, 0, 0, 0, 0]",This five-year project (September 2016 – Augus...,Four major inter-related causes of conflict an...,"(By pooling resources and expertise\, the proj..."
4,36,"[PROJECT OBJECTIVES:, nnThe objective of the p...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",PROJECT OBJECTIVES:\n\nThe objective of the pr...,The objective of the project is to increase ba...,(PROJECT OBJECTIVES:\n\nThe objective of the p...
...,...,...,...,...,...,...
569,2967,[Money Trail is a joint project initiated by F...,"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]",Money Trail is a joint project initiated by Fr...,Money Trail is a joint project initiated by Fr...,(Finance Uncovered provides training in financ...
570,2973,"[Gisha, Legal Center for Freedom of Movement i...","[0, 0, 0, 1, 0, 0, 0, 0]","Gisha\, Legal Center for Freedom of Movement i...",Gisha#s direct intervention with Palestinians ...,(Of particular added value is the existing est...
571,2976,[Legal Aid Service Providers Network Uganda (L...,"[0, 1, 1, 0, 0, 0, 0]",Legal Aid Service Providers Network Uganda (LA...,"this Voice linking and learning project, LASP...","(On this Voice linking and learning project\, ..."
572,2977,[The latest findings from a countrywide season...,"[0, 0, 0, 0, 1, 0, 0, 0]",The latest findings from a countrywide seasona...,nOxfam provides integrated EFSVL (Emergency Fo...,(Water shortage may also trigger further outbr...


# 4. Evaluation

## 4.1. Data Preparation

In [12]:
# Creating lists of the reference and the predicted summaries
reference_summaries = df_summaries['reference_summary']
predicted_summaries = df_summaries['predicted_summary']

# Converting the predicted summaries to strings 
predicted_summaries = [str(sent) for sent in predicted_summaries]

# Defining a function to clean the predicted sentences from the added format generated by text_rank
def cleaner(sent):
  sent = sent.replace("(<Sentence: ", "")
  sent = sent.replace(" <Sentence:", "")
  sent = sent.replace ('>)', "")
  sent = sent.replace('>,', "")
  return sent 

# Applying the function to both the predicted and the reference summaries for consistency
predicted_summaries = [cleaner(sent) for sent in predicted_summaries]
reference_summaries = [cleaner(sent) for sent in reference_summaries]

## 4.2. Computing the ROUGE scores

In [13]:
# Loading the ROUGE metric from Hugginface 
metric = load_metric('rouge')
rouge_scores = metric.compute(predictions=predicted_summaries, references=reference_summaries)
rouge_scores

Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

{'rouge1': AggregateScore(low=Score(precision=0.23537639054245538, recall=0.7226346100188772, fmeasure=0.34174465334571974), mid=Score(precision=0.24836741741475, recall=0.7437164899385587, fmeasure=0.3581034267337886), high=Score(precision=0.261714155727758, recall=0.7639330778064907, fmeasure=0.37538562028420214)),
 'rouge2': AggregateScore(low=Score(precision=0.17336583639578926, recall=0.5104787803913226, fmeasure=0.24919076867912915), mid=Score(precision=0.18868075592488723, recall=0.5461793421396745, fmeasure=0.26961201993371514), high=Score(precision=0.20465325408251725, recall=0.579087055635583, fmeasure=0.2904257988873584)),
 'rougeL': AggregateScore(low=Score(precision=0.21006395027770192, recall=0.6346949008325515, fmeasure=0.30399968108266373), mid=Score(precision=0.22298619134148187, recall=0.6594833779574482, fmeasure=0.32033712152906246), high=Score(precision=0.23677232289075645, recall=0.6851139852138847, fmeasure=0.3381639796490175)),
 'rougeLsum': AggregateScore(low=S

## 4.3. View a few examples

In [ ]:
print('This is the reference summary: ')
print(reference_summaries[16])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[16])

This is the reference summary: 
This project contributes to the financing of the expansion of the Koudougou university campus consisting of the construction and the equipment of new facilities
   
This is the predicted summary: 
The substantial increase in the number of new admitted students each year reached about 5,000, who enrol for the first time in UoK and represents a huge challenge for the University to accommodate them. Hence, in April 2017, the government requested OFID to contribute to the financing of the expansion of the university campus consisting of the construction and the equipment of new facilities including: a) Dormitories, b) Science and Technology Training and Academic Research Unit.


In [ ]:
print('This is the reference summary: ')
print(reference_summaries[69])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[69])

This is the reference summary: 
The development objective of the Remote Rural Communities Development Project for Bhutan is to increase agriculture productivity and access to community assets in remote rural areas by improving access to markets, irrigation, agricultural technologies, and community infrastructure.
   
This is the predicted summary: 
The development objective of the Remote Rural Communities Development Project for Bhutan is to increase agriculture productivity and access to community assets in remote rural areas by improving access to markets, irrigation, agricultural technologies, and community infrastructure. This component will also support midterm and end-of-project project monitoring surveys that would provide project performance information at the activity, output, and outcome levels to meet the requirements of the existing Royal Government of Bhutan (RGoB) monitoring system and the Planning and Monitoring System (PLAMS).


In [ ]:
print('This is the reference summary: ')
print(reference_summaries[103])
print( '   ')
print('This is the predicted summary: ')
print(predicted_summaries[103])

This is the reference summary: 
The development objective Casablanca Municipal Support Program-for-Results Project for Morocco is to increase the investment capacity of the Municipality of Casablanca, improve the business environment in the Municipality of Casablanca, and enhance access to basic services in the Program Area. 
   
This is the predicted summary: 
The Program activities are grouped under three Results Areas (RA): (i) RA1: Increasing the investment capacity of the Municipality of Casablanca; (ii) RA2: Improving the urban environment and access to basic services in the Program Area; and (iii) RA3: Improving the business environment in the Municipality of Casablanca. Specifically, the operation will support two of the CPS strategic result areas, namely: (i) Result Area 1: Promoting Competitive and Inclusive Growth, by improving the business environment in a major metropolitan area constituting the country's main engine of growth; and (ii) Result Area 3: Strengthening Governa